In [15]:
#Prediction of mobilenet_student_distilled_1.pth model

import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np

# Load the model architecture
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)  # Binary classification
model.load_state_dict(torch.load("mobilenet_student_distilled_1.pth", map_location='cpu'))
model.eval()

# Transform (same as used during training)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Prediction function
def predict(image_path, threshold=0.85):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1).squeeze()
        pred_class = torch.argmax(probs).item()
        recaptured_prob = probs[1].item()
        flag = 1 if recaptured_prob >= threshold else 0
    return flag, recaptured_prob, pred_class

# Example usage
image_path = r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\Tenserflow\dfsjh.jpeg"
flag, recaptured_prob, pred_class = predict(image_path)
print(f"Flag: {flag}, Probability: {recaptured_prob:.4f}, Predicted Class: {pred_class}")


Flag: 1, Probability: 0.9832, Predicted Class: 1


c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\nnms_fastai\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\nnms_fastai\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [16]:
# Convert mobilenet_student_distilled_1.pth into mobilenet_student_distilled.pkl
from fastai.vision.all import *
from torchvision import models
import torch

# Path to your image dataset
data_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Image_classification\PMAY-PHOTO OF PHOTO\PMAY-PHOTO OF PHOTO\training_test_data\data_v4\train")

# Load dls (DataLoaders)
dls = ImageDataLoaders.from_folder(
    data_path.parent,  # Parent folder containing 'train' and 'valid' if available
    train='train',
    valid_pct=0.2,
    item_tfms=Resize(256),
    batch_tfms=Normalize.from_stats(*imagenet_stats),
    bs=32
)

# Define the model architecture
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)
model.load_state_dict(torch.load("mobilenet_student_distilled_1.pth", map_location='cpu'))

# Create fastai Learner using your model and dls
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)

# Export
learn.export("mobilenet_student_distilled.pkl")
print("Exported successfully.")


Exported successfully.


In [17]:
#prediction on mobilenet_student_distilled.pkl model
learn = load_learner("mobilenet_student_distilled.pkl")
learn.predict(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\Tenserflow\dfsjh.jpeg")


('recaptured', tensor(1), tensor([0.1055, 0.8945]))

In [18]:
from torchvision.models import mobilenet_v2
import torch

# Step 1: Load the state_dict or full model saved with torch.save()
loaded_model = torch.load('mobilenet_student_distilled.pkl', map_location='cpu')

# Step 2: Initialize a MobileNetV2 model
mobilenet_model = mobilenet_v2()

# Step 3: Update the classifier for 2 output classes
mobilenet_model.classifier[1] = torch.nn.Linear(mobilenet_model.last_channel, 2)

# Step 4: Load weights
if isinstance(loaded_model, dict):
    mobilenet_model.load_state_dict(loaded_model, strict=False)
else:
    mobilenet_model.load_state_dict(loaded_model.state_dict(), strict=False)

# Step 5: Save the state_dict as a .pt file
torch.save(mobilenet_model.state_dict(), 'mobilenet_distilled_v2.pt')

print("Successfully converted and saved as mobilenet_distilled_v2.pt")


Successfully converted and saved as mobilenet_distilled_v2.pt


In [20]:
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models import mobilenet_v2

# Step 1: Define the model architecture
model = mobilenet_v2()
model.classifier[1] = torch.nn.Linear(model.last_channel, 2)  # Match output classes
model.load_state_dict(torch.load("mobilenet_distilled_v2.pt", map_location='cpu'))
model.eval()

# Step 2: Preprocess the image
image = Image.open(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\Classification\Tenserflow\dfsjh.jpeg").convert("RGB")
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
input_tensor = preprocess(image).unsqueeze(0)  # Shape: (1, 3, 224, 224)

# Step 3: Make prediction
with torch.no_grad():
    output = model(input_tensor)
pred = torch.argmax(output, dim=1).item()

# Step 4: (Optional) Map index to class label
class_names = ['class_0', 'class_1']  # Replace with your actual class labels
print("Predicted class index:", pred)
print("Predicted class label:", class_names[pred])


Predicted class index: 1
Predicted class label: class_1
